# Bulk download fic pdfs

In [105]:
import os
import json
import re
import sys
sys.path.append("..")
import requests
import pandas as pd
from time import time
import logging
from multiprocessing.pool import ThreadPool

In [2]:
pwd

'/Users/yuchen.zhang/Documents/Projects/Class_Action/class-action-git/src'

In [3]:
# Load input data
scraped_data = pd.read_csv("../../data/data_from_scrapy.csv")

In [4]:
# Example fic summary table
pd.read_json(scraped_data["fic_summary_table"][0])

,No,Document Title,Filing Date,link
0,1,Complaint - Class Action,07/07/2020,http://securities.stanford.edu/filings-documen...
1,2,U.S. District Court Civil Docket,07/08/2020,http://securities.stanford.edu/filings-documen...


In [93]:
# Set Output Folder location
output_folder = "/Volumes/yz-drive/class-action-complaints"

In [94]:
# Function to download a pdf from a link
def url_response(url):
    path, url = url
    r = requests.get(url, stream = True)
    
    with open(output_folder + "/" + path,"wb") as pdf: 
        for chunk in r.iter_content(chunk_size=1024): 

            # writing one chunk at a time to pdf file 
            if chunk: 
                 pdf.write(chunk)

In [269]:
# Function to bulk download
def downloadall(data_raw):
    #remove entry where fic_summary_table is not found
    data = data_raw[data_raw['fic_summary_table'].notna()]
    #a new dictionary for each docket and document
    documentDic = {}
    
    #loop through data (scraped data)
    for x,y in data.iterrows():
        #save subtable
        table = pd.read_json(data["fic_summary_table"][x])
        print(x)
        
        #save docket
        docket = data["docket"][x]
        if pd.isnull(docket):
            docket = "not found"
        
        #loop through each document in fic_summary_table
        for i, j in table.iterrows():
            
            #save title and date to be parts of the dictionary and file name
            title = table["Document Title"][i][0:200].replace("/","")
            date = table["Filing Date"][i].replace("/","")
            title_date = title + "_" + date
            
            if docket not in documentDic:
                documentDic[docket] = [title_date]
            else:
                documentDic = documentDic[docket] + [title_date]

            filename = docket + "_" + title + "_" + date + ".pdf"
            print(filename)
            url_response([filename, table["link"][i]])

In [271]:
#downloadall(scraped_data)